In [28]:
# !pip install nlp_primitives
# !pip install bs4
# !pip install spacy
# !pip install gensim
# !pip install nltk
# !pip install seaborn
# ! python -m spacy download en

In [16]:
# from google.colab import drive
# drive.mount("/content/drive", force_remount=True)

In [83]:
import pandas as pd
import numpy as np
import os
import re
import zipfile
import requests
from bs4 import BeautifulSoup
from collections import Counter
import itertools

import spacy
from spacy.tokenizer import Tokenizer
import gensim as gen
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import featuretools as ft
import featuretools.variable_types as vtypes
from featuretools.primitives import  AggregationPrimitive
from nlp_primitives import (
    DiversityScore,
    LSA,
    MeanCharactersPerWord,
    PolarityScore, 
    UniversalSentenceEncoder,
    PunctuationCount,
    StopwordCount,
    TitleWordCount,
    UpperCaseCount)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
AMBIGUOUS_WORDS_URL = 'https://muse.dillfrog.com/lists/ambiguous'

Path

In [23]:
# yaniv
# path = "/content/drive/My Drive/Final Project (Personal)/"

# avi
# path = "/content/drive/My Drive/DataScientistITCAvi/Final_Project/Model/Notebooks/"

# cnvrg
path = ""

Load Dataset:

In [24]:
df = pd.read_pickle(path+'eda_reddit_jokes.pkl')[:5000]

In [25]:
df.head()

,joke,score
0,I hate how you cant even say black paint anymo...,1
1,"Brian raises his hand and says, “He’s in Heave...",1
2,I walked into a PETA adoption center and the r...,1
3,Remember when you were a kid and when you crie...,15
4,"My boss said to me, ""you're the worst train dr...",3


Load Gensim word embedddings (Google news negative) - Need to check if there's a better embedding to use

In [ ]:
# model_embeddings = gen.models.KeyedVectors.load_word2vec_format(path+'GoogleNews-vectors-negative300.bin',binary=True)

In [29]:
nlp = spacy.load('en')
def en_tokenize(string):
    return nlp(string)

Add ambiguous words counter features:

In [30]:
def get_ambiguous_words():
  res = requests.get(AMBIGUOUS_WORDS_URL)
  page_soup = BeautifulSoup(res.content)
  a_tags = page_soup.find_all('a', href=re.compile(r'.*/meaning/word/*'))
  ambiguous_words = [word.text for word in a_tags]
  return ambiguous_words

In [31]:
ambiguous_words = get_ambiguous_words()
df['ambiguous_words'] = df['joke'].apply(lambda x: sum([str(w) in ambiguous_words for w in x]))
df['ambiguous_words'].head()

0    4
1    7
2    3
3    2
4    1
Name: ambiguous_words, dtype: int64

Repetition

In [32]:
spacy_col = df['joke'].apply(en_tokenize)

In [33]:
token_pos = spacy_col.apply(lambda x: [(elm.text, elm.pos_) for elm in x])

In [34]:
from gensim.models import Word2Vec
from nltk.corpus import brown
nltk.download('brown')
b = Word2Vec(brown.sents())

def find_best_similarity(string):
    """ find the max similarity between two words """
    # unique words:
    word_list = list(set([word_pos[0] for word_pos in string if word_pos[1] != "PROPN"]))
    all_combi = list(itertools.combinations(word_list, 2))

    max_similarity = 0
    for pairs in all_combi:
        try:
            temp_similatity = b.similarity(pairs[0], pairs[1])
            if temp_similatity > max_similarity:
                max_similarity = temp_similatity
        except:
            pass # don't found the word in word2vec
    return max_similarity

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [35]:
df['best_score_similarity_words'] = token_pos.apply(find_best_similarity)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  from ipykernel import kernelapp as app


In [36]:
df['best_score_similarity_words'].head()

0    0.919720
1    0.945242
2    0.932165
3    0.912641
4    0.914353
Name: best_score_similarity_words, dtype: float64

Antoym

In [37]:
list_of_antonyms=[]
from nltk.corpus import wordnet as wn
for i in wn.all_synsets():
    if i.pos() in ['a', 's']: 
        for j in i.lemmas(): 
            if j.antonyms(): 
                (j.name(), j.antonyms()[0].name()) and list_of_antonyms.append((j.name(), j.antonyms()[0].name()))
dict_antonyms = dict((y, x) for x, y in list_of_antonyms)

def find_antonyms(tokens_poss):
    """ count how many antonyms in sentence """
    word_list = [word_pos[0] for word_pos in tokens_poss]
    count = 0
    for word in word_list:
        try:
            if word in dict_antonyms:
                if dict_antonyms[word] in word_list:
                    count += 1
        except Exception:
            pass # don't found in dict
    return count


In [38]:
df['antonyms'] = token_pos.apply(find_antonyms)

In [39]:
df['antonyms'].head()

0    0
1    0
2    0
3    0
4    0
Name: antonyms, dtype: int64

Longest word

In [40]:
def len_longest_word(tokens_poss):
    """ check the length of the longest word """
    word_len_list = [(word_pos[0], len(word_pos[0]))  for word_pos in tokens_poss]
    word_len_list = sorted(word_len_list, key=lambda x: x[1])

    return word_len_list[-1][1]

In [41]:
df['longest_word'] = token_pos.apply(len_longest_word)

In [42]:
df['longest_word'].head()

0     7
1     9
2    12
3     8
4     8
Name: longest_word, dtype: int64

How many speical chars

In [43]:
def count_speical_chars(tokens_poss):
    """ how many speical chars in string """
    bin_isalphanumberic_list = [not word_pos[0].isalnum() for word_pos in tokens_poss]

    return np.sum(bin_isalphanumberic_list)

In [44]:
df['speical_chars'] = token_pos.apply(count_speical_chars)

In [45]:
df['speical_chars'].head()

0     4
1    44
2     5
3     8
4    14
Name: speical_chars, dtype: int64

more feaures from "featuretools" package.

In [46]:
es = ft.EntitySet("jokes_df")
es.entity_from_dataframe(entity_id="jokes_df",
                          index="joke_id",
                          make_index=True,
                          dataframe=df)

Entityset: jokes_df
  Entities:
    jokes_df [Rows: 5000, Columns: 8]
  Relationships:
    No relationships

In [84]:
"""
DiversityScore()
Calculates the overall complexity of the text based on the total

LSA()
Calculates the Latent Semantic Analysis Values of Text Input

MeanCharactersPerWord()
Determines the mean number of characters per word.

PolarityScore()
Calculates the polarity of a text on a scale from -1 (negative) to 1 (positive)

PunctuationCount()
Determines number of punctuation characters in a string.

StopwordCount()
Determines number of stopwords in a string.

TitleWordCount()
Determines the number of title words in a string.

UpperCaseCount()
Calculates the number of upper case letters in text.
"""

trans = [
#     DiversityScore,
#          LSA,
         MeanCharactersPerWord,
         UniversalSentenceEncoder,
         PolarityScore, 
         PunctuationCount,
         StopwordCount,
         TitleWordCount,
         UpperCaseCount]

In [85]:
feature_matrix_customers, features_defs = ft.dfs(entityset=es,
                  target_entity='jokes_df',
                #   instance_ids=["joke"],
                  trans_primitives=trans,
                  max_depth=4)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [86]:
# number of features to add:
len(features_defs)

13

In [87]:
df = pd.concat([df['joke'],feature_matrix_customers], axis=1)

In [88]:
df.head()

,joke,score,ambiguous_words,best_score_similarity_words,antonyms,longest_word,speical_chars,MEAN_CHARACTERS_PER_WORD(joke),UNIVERSAL_SENTENCE_ENCODER(joke)[0],UNIVERSAL_SENTENCE_ENCODER(joke)[1],...,UNIVERSAL_SENTENCE_ENCODER(joke)[507],UNIVERSAL_SENTENCE_ENCODER(joke)[508],UNIVERSAL_SENTENCE_ENCODER(joke)[509],UNIVERSAL_SENTENCE_ENCODER(joke)[510],UNIVERSAL_SENTENCE_ENCODER(joke)[511],POLARITY_SCORE(joke),PUNCTUATION_COUNT(joke),STOPWORD_COUNT(joke),TITLE_WORD_COUNT(joke),UPPER_CASE_COUNT(joke)
0,I hate how you cant even say black paint anymo...,1,4,0.919720,0,7,4,3.863636,0.015197,-0.033341,...,-0.057825,0.019375,0.041815,-0.047018,-0.016727,-0.083,4.0,7,4.0,4.0
1,"Brian raises his hand and says, “He’s in Heave...",1,7,0.945242,0,9,44,4.584158,-0.041704,0.062440,...,-0.068055,-0.035025,-0.039353,-0.045778,-0.006202,0.195,20.0,43,25.0,25.0
2,I walked into a PETA adoption center and the r...,1,3,0.932165,0,12,5,4.750000,-0.033150,0.027067,...,-0.072915,0.020479,0.054477,-0.046195,0.033442,0.079,5.0,11,5.0,8.0
3,Remember when you were a kid and when you crie...,15,2,0.912641,0,8,8,4.073171,0.000824,0.058427,...,-0.054912,0.017778,-0.040058,-0.023789,-0.023233,-0.276,8.0,20,3.0,3.0
4,"My boss said to me, ""you're the worst train dr...",3,1,0.914353,0,8,14,4.035714,-0.066581,0.072912,...,-0.071312,-0.002545,-0.024180,-0.037230,-0.007027,-0.345,14.0,10,4.0,4.0


Add object counter features:

In [91]:
df['joke_tokenized'] = df['joke'].apply(nlp)

In [92]:
def add_object_count_cols(df,tokenized_col):
  df['temp'] = df[tokenized_col].apply(lambda x: [ent.label_ for ent in x.ents])
  mlb = MultiLabelBinarizer()
  mlb.fit(df['temp'])
  df = df.join(pd.DataFrame(mlb.transform(df['temp']),
                            columns=mlb.classes_,
                            index=df.index))
  del df['temp']
  return df


In [93]:
df = add_object_count_cols(df,'joke_tokenized')

In [94]:
df.shape

(5000, 544)

Add Part-of-Speech features:

In [95]:
def add_pos_count_cols(df,tokenized_col):
  df['temp'] = df[tokenized_col].apply(lambda x: [ent.pos_ for ent in x])
  mlb = MultiLabelBinarizer()
  mlb.fit(df['temp'])
  df = df.join(pd.DataFrame(mlb.transform(df['temp']),
                            columns=mlb.classes_,
                            index=df.index))
  del df['temp']
  return df

In [96]:
df = add_pos_count_cols(df,'joke_tokenized')

In [97]:
df.shape

(5000, 562)

Text cleaning:

In [98]:
def replace_non_eng_punct(txt):
    return re.sub(r'/[^a-zA-Z0-9\s,.?!]/','*',txt).strip()

In [99]:
def replace_escape(txt):
    updated_txt = re.sub(r'\n|\t|&amp;',' ',txt)
    return updated_txt.strip()

In [100]:
def remove_multi_spaces(txt):
    return re.sub(' +', ' ',txt)

In [101]:
def preprocess_text(document):
  #         # Remove all the special characters
  document = re.sub(r'\W', ' ', str(document))

  #         # remove all single characters
  document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

  #         # Remove single characters from the start
  document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

  # Substituting multiple spaces with single space
  document = re.sub(r'\s+', ' ', document, flags=re.I)

  # Removing prefixed 'b'
  document = re.sub(r'^b\s+', '', document)

  # Converting to Lowercase
  document = document.lower()

  return document

In [102]:
df['joke_text_processed'] = df['joke'].apply(replace_non_eng_punct).apply(remove_multi_spaces).apply(replace_escape)

In [103]:
df['joke_text_processed'] = df['joke_text_processed'].apply(preprocess_text)

In [104]:
df['joke_processed_tokenized'] = df['joke_text_processed'].apply(nlp)

In [105]:
df['joke_processed_tokenized'][55]

did you hear about leatherface jewish cousin he was also serial killer he liked to dig up the corpses of women and use their skin to furnish his house after the police arrested him they discovered whole morbid collection of objects he had belt made out of ears lampshade made from stitched together faces they even found labia menora 

Lemmatization:

In [106]:
stemmer = WordNetLemmatizer()
stop_words = stopwords.words('english')
lemmatize_remove_stop_words = lambda x: nlp(' '.join([stemmer.lemmatize(str(word)) for word in x if str(word) not in stop_words]))
lemmatize_remove_stop_words_str = lambda x: ' '.join([stemmer.lemmatize(str(word)) for word in x if str(word) not in stop_words])

In [107]:
df['joke_processed_tokenized_stemmed'] = df['joke_processed_tokenized'].apply(lemmatize_remove_stop_words)
df['joke_processed_tokenized_stemmed_str'] = df['joke_processed_tokenized'].apply(lemmatize_remove_stop_words_str)

Build embedding features:

In [ ]:
# def get_sentence_embeddings(sentence):
#   embedding = np.zeros((len(sentence),300))
#   for ind,token in enumerate(sentence):
#     if str(token) in model_embeddings.vocab.keys():
#       embedding[ind] = model_embeddings[str(token)]
#     else:
#       pass
#   embedding = embedding.mean(axis = 0)
#   return embedding

# def add_pos_count_cols(df,tokenized_col):
#   df['temp'] = df[tokenized_col].apply(lambda x: [ent.pos_ for ent in x])
#   mlb = MultiLabelBinarizer()
#   mlb.fit(df['temp'])
#   df = df.join(pd.DataFrame(mlb.transform(df['temp']),
#                             columns=mlb.classes_,
#                             index=df.index))

In [ ]:
# df['joke_embbedings'] = df['joke_processed_tokenized_stemmed'].apply(get_sentence_embeddings)

In [ ]:
# classes = [f'sentence_vec_{i}' for i in range(300)]
# df = df.merge(pd.DataFrame(df['joke_embbedings'].tolist(), columns = classes, index= df['joke_embbedings'].index),
#                            how='left',left_index=True, right_index=True)

In [108]:
df.shape

(5000, 566)

Build total words / total chars features:

In [109]:
df['total_words'] = df['joke_tokenized'].apply(len)
df['total_chars'] = df['joke'].apply(len)

Split to train/test set:

In [186]:
target = 'score'
X = df[df.columns[df.columns!=target]]
y = df[target]

In [187]:
## Use validation set
# Split earlier to avoid leakage
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [189]:
X_train.reset_index(inplace = True)
X_test.reset_index(inplace = True)
y_train = y_train.reset_index()['score']
y_test = y_test.reset_index()['score']

In [192]:
X_test.shape

(1000, 568)

In [193]:
X_train = X_train.loc[:,X_train.columns != 'index']
X_test = X_test.loc[:,X_test.columns != 'index']

In [145]:
vectorizer = TfidfVectorizer(input='content', decode_error='strict', lowercase=True, stop_words='english', ngram_range=(1, 2), 
                max_features=1000, binary=True)

Add TfidfVectorizer features:

In [194]:
vectorizer.fit(X_train['joke_processed_tokenized_stemmed_str'])
tfidf_vec_train = vectorizer.transform(X_train['joke_processed_tokenized_stemmed_str']).toarray()
tfidf_vec_test = vectorizer.transform(X_test['joke_processed_tokenized_stemmed_str']).toarray()

In [195]:
X_train = X_train.merge(pd.DataFrame(tfidf_vec_train,columns=vectorizer.vocabulary_),how='left',left_index=True,right_index=True)
X_test = X_test.merge(pd.DataFrame(tfidf_vec_test,columns=vectorizer.vocabulary_),how='left',left_index=True,right_index=True)

Check Missing Data:

In [197]:
def check_missing_data(df):
  """ 
      params: dataframe
      return: data of missing values, if have some.
  """
  flag=df.isna().sum().any()
  if flag==True:
      total = df.isnull().sum()
      percent = (df.isnull().sum())*100/(df.isnull().count())
      output = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
      data_type = []
      for col in df.columns:
          dtype = str(df[col].dtype)
          data_type.append(dtype)
      output['Types'] = data_type
      return(output[output['Percent'] > 0].sort_values(by='Percent', ascending=False))
  else:
      return(False)

Dump output:

In [118]:
pd.to_pickle(X_train, path + "X_train.pkl")
pd.to_pickle(X_test, path + "X_test.pkl")
pd.to_pickle(y_train, path + "y_train.pkl")
pd.to_pickle(y_test, path + "y_test.pkl")